In [177]:
import pandas as pd
import numpy as np

In [178]:
df = pd.read_csv('dataframe_20k_290623.csv').drop(columns=['Unnamed: 0'])

In [179]:
df.head()

,locality,type of property,subtype of property,price,number_rooms,living_area,kitchen,furnished,fireplace,terrace,terrace_area,garden,garden_area,surface_land,number_facades,swimming_pool,building_state
0,Ham,HOUSE,VILLA,429500.0,4.0,215.0,INSTALLED,NaN,-1.0,NaN,NaN,True,NaN,1472,4,NaN,GOOD
1,Putte,HOUSE,HOUSE,599000.0,5.0,167.0,INSTALLED,NaN,-1.0,True,NaN,True,95.0,416,4,NaN,JUST_RENOVATED
2,IVOZ-RAMET,HOUSE,HOUSE,20000.0,3.0,130.0,INSTALLED,NaN,-1.0,True,NaN,NaN,NaN,182,3,NaN,GOOD
3,Herstal,HOUSE,HOUSE,45000.0,2.0,120.0,USA_INSTALLED,True,NaN,True,12.0,True,80.0,221,2,False,TO_BE_DONE_UP
4,Liege,HOUSE,HOUSE,109000.0,2.0,62.0,SEMI_EQUIPPED,False,NaN,NaN,NaN,NaN,NaN,45,2,False,TO_BE_DONE_UP


In [180]:
df.rename(columns = {'type of property':'property_type', 'subtype of property':'property_subtype'}, inplace = True)
df.head()

,locality,property_type,property_subtype,price,number_rooms,living_area,kitchen,furnished,fireplace,terrace,terrace_area,garden,garden_area,surface_land,number_facades,swimming_pool,building_state
0,Ham,HOUSE,VILLA,429500.0,4.0,215.0,INSTALLED,NaN,-1.0,NaN,NaN,True,NaN,1472,4,NaN,GOOD
1,Putte,HOUSE,HOUSE,599000.0,5.0,167.0,INSTALLED,NaN,-1.0,True,NaN,True,95.0,416,4,NaN,JUST_RENOVATED
2,IVOZ-RAMET,HOUSE,HOUSE,20000.0,3.0,130.0,INSTALLED,NaN,-1.0,True,NaN,NaN,NaN,182,3,NaN,GOOD
3,Herstal,HOUSE,HOUSE,45000.0,2.0,120.0,USA_INSTALLED,True,NaN,True,12.0,True,80.0,221,2,False,TO_BE_DONE_UP
4,Liege,HOUSE,HOUSE,109000.0,2.0,62.0,SEMI_EQUIPPED,False,NaN,NaN,NaN,NaN,NaN,45,2,False,TO_BE_DONE_UP


In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19980 entries, 0 to 19979
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   locality          19974 non-null  object 
 1   property_type     19974 non-null  object 
 2   property_subtype  19974 non-null  object 
 3   price             19499 non-null  float64
 4   number_rooms      19499 non-null  float64
 5   living_area       18445 non-null  float64
 6   kitchen           18077 non-null  object 
 7   furnished         9815 non-null   object 
 8   fireplace         6176 non-null   float64
 9   terrace           12887 non-null  object 
 10  terrace_area      7391 non-null   float64
 11  garden            9078 non-null   object 
 12  garden_area       5916 non-null   float64
 13  surface_land      18209 non-null  object 
 14  number_facades    16438 non-null  object 
 15  swimming_pool     10256 non-null  object 
 16  building_state    16589 non-null  object

In [182]:
df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19980 entries, 0 to 19979
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   locality          19974 non-null  object 
 1   property_type     19974 non-null  object 
 2   property_subtype  19974 non-null  object 
 3   price             19499 non-null  float64
 4   number_rooms      19499 non-null  float64
 5   living_area       18445 non-null  float64
 6   kitchen           18077 non-null  object 
 7   furnished         9815 non-null   object 
 8   fireplace         6176 non-null   float64
 9   terrace           12887 non-null  object 
 10  terrace_area      7391 non-null   float64
 11  garden            9078 non-null   object 
 12  garden_area       5916 non-null   float64
 13  surface_land      18209 non-null  object 
 14  number_facades    16438 non-null  object 
 15  swimming_pool     10256 non-null  object 
 16  building_state    16589 non-null  object

In [183]:
def analyze_column(df, column_name):
    column = df[column_name]
    num_nan = column.isnull().sum()
    num_unique = column.nunique()
    unique_values = column.unique()
    percentage_nan = (num_nan / len(column)) * 100
    percentage_unique = (num_unique / len(column)) * 100
    print(f"Analysis for column '{column_name}':")
    print(f"Number of NaN values: {num_nan}")
    print(f"Number of unique values: {num_unique}")
    print(f"Percentage of NaN values: {percentage_nan:.2f}%")
    print(f"Percentage of unique values: {percentage_unique:.2f}%")
    print(f"Unique values: {unique_values}")

In [184]:
df = df.dropna(how='all')

### 0. PRICE

In [185]:
# If we dont have a price - Delete the rows - not useable to predict pricing!

In [186]:
# This will be what we will need to predict
df['price'].info()
df.price.info()

<class 'pandas.core.series.Series'>
Index: 19974 entries, 0 to 19979
Series name: price
Non-Null Count  Dtype  
--------------  -----  
19499 non-null  float64
dtypes: float64(1)
memory usage: 312.1 KB
<class 'pandas.core.series.Series'>
Index: 19974 entries, 0 to 19979
Series name: price
Non-Null Count  Dtype  
--------------  -----  
19499 non-null  float64
dtypes: float64(1)
memory usage: 312.1 KB


In [187]:
# We have 475 rows with NaN as price - 2.38% of the total

analyze_column(df, 'price')

Analysis for column 'price':
Number of NaN values: 475
Number of unique values: 1624
Percentage of NaN values: 2.38%
Percentage of unique values: 8.13%
Unique values: [429500. 599000.  20000. ... 381500. 351000. 176000.]


In [188]:
df[df['price'].isna()]
# Most values are not Filled in (NaN) so I decide to drop all of these 475 rows

,locality,property_type,property_subtype,price,number_rooms,living_area,kitchen,furnished,fireplace,terrace,terrace_area,garden,garden_area,surface_land,number_facades,swimming_pool,building_state
47,Roeselare,APARTMENT_GROUP,APARTMENT_GROUP,NaN,NaN,NaN,0,False,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,NaN,UNKNOWN
90,Evere,APARTMENT_GROUP,APARTMENT_GROUP,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,NaN,UNKNOWN
181,Balen,HOUSE_GROUP,HOUSE_GROUP,NaN,NaN,NaN,0,False,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,NaN,UNKNOWN
182,HERSELT,HOUSE_GROUP,HOUSE_GROUP,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,NaN,UNKNOWN
183,WITGOOR,HOUSE_GROUP,HOUSE_GROUP,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,NaN,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18901,Mons,HOUSE_GROUP,HOUSE_GROUP,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,NaN,UNKNOWN
19186,Borgloon,HOUSE_GROUP,HOUSE_GROUP,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,NaN,UNKNOWN
19537,Borgloon,APARTMENT_GROUP,APARTMENT_GROUP,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,NaN,UNKNOWN
19718,Dessel,HOUSE_GROUP,HOUSE_GROUP,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,NaN,UNKNOWN


In [189]:
df = df.dropna(subset=['price'])

In [190]:
df[df['price'].isna()]
# ALL NaN Values are dropped

,locality,property_type,property_subtype,price,number_rooms,living_area,kitchen,furnished,fireplace,terrace,terrace_area,garden,garden_area,surface_land,number_facades,swimming_pool,building_state


In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19499 entries, 0 to 19979
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   locality          19499 non-null  object 
 1   property_type     19499 non-null  object 
 2   property_subtype  19499 non-null  object 
 3   price             19499 non-null  float64
 4   number_rooms      19499 non-null  float64
 5   living_area       18445 non-null  float64
 6   kitchen           17602 non-null  object 
 7   furnished         9756 non-null   object 
 8   fireplace         6176 non-null   float64
 9   terrace           12887 non-null  object 
 10  terrace_area      7391 non-null   float64
 11  garden            9078 non-null   object 
 12  garden_area       5916 non-null   float64
 13  surface_land      17734 non-null  object 
 14  number_facades    15963 non-null  object 
 15  swimming_pool     10256 non-null  object 
 16  building_state    16114 non-null  object 
dty

In [192]:
# Resetting index to have nice looking dataframe
df.reset_index(inplace=True, drop=True)

In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19499 entries, 0 to 19498
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   locality          19499 non-null  object 
 1   property_type     19499 non-null  object 
 2   property_subtype  19499 non-null  object 
 3   price             19499 non-null  float64
 4   number_rooms      19499 non-null  float64
 5   living_area       18445 non-null  float64
 6   kitchen           17602 non-null  object 
 7   furnished         9756 non-null   object 
 8   fireplace         6176 non-null   float64
 9   terrace           12887 non-null  object 
 10  terrace_area      7391 non-null   float64
 11  garden            9078 non-null   object 
 12  garden_area       5916 non-null   float64
 13  surface_land      17734 non-null  object 
 14  number_facades    15963 non-null  object 
 15  swimming_pool     10256 non-null  object 
 16  building_state    16114 non-null  object

### 1. LOCALITY

In [201]:
analyze_column(df, 'locality')

Analysis for column 'locality':
Number of NaN values: 0
Number of unique values: 2589
Percentage of NaN values: 0.00%
Percentage of unique values: 13.28%
Unique values: ['Ham' 'Putte' 'IVOZ-RAMET' ... 'Louveigné' 'GrÃce-Hollogne' 'Schoonaarde']


In [202]:
df['locality'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 19499 entries, 0 to 19498
Series name: locality
Non-Null Count  Dtype 
--------------  ----- 
19499 non-null  object
dtypes: object(1)
memory usage: 152.5+ KB


### 2. Property Type 

In [203]:
analyze_column(df, 'property_type')

Analysis for column 'property_type':
Number of NaN values: 0
Number of unique values: 2
Percentage of NaN values: 0.00%
Percentage of unique values: 0.01%
Unique values: ['HOUSE' 'APARTMENT']


In [204]:
# LOOKS OK, NO NAN Values
df['property_type'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 19499 entries, 0 to 19498
Series name: property_type
Non-Null Count  Dtype 
--------------  ----- 
19499 non-null  object
dtypes: object(1)
memory usage: 152.5+ KB


### 3. Property Subtype

In [198]:
analyze_column(df, 'property_subtype')

Analysis for column 'property_subtype':
Number of NaN values: 0
Number of unique values: 23
Percentage of NaN values: 0.00%
Percentage of unique values: 0.12%
Unique values: ['VILLA' 'HOUSE' 'APARTMENT_BLOCK' 'CHALET' 'COUNTRY_COTTAGE' 'APARTMENT'
 'FLAT_STUDIO' 'MIXED_USE_BUILDING' 'EXCEPTIONAL_PROPERTY' 'TOWN_HOUSE'
 'PENTHOUSE' 'DUPLEX' 'GROUND_FLOOR' 'MANSION' 'BUNGALOW' 'OTHER_PROPERTY'
 'CASTLE' 'SERVICE_FLAT' 'LOFT' 'KOT' 'TRIPLEX' 'FARMHOUSE' 'MANOR_HOUSE']


In [199]:
df['property_subtype'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 19499 entries, 0 to 19498
Series name: property_subtype
Non-Null Count  Dtype 
--------------  ----- 
19499 non-null  object
dtypes: object(1)
memory usage: 152.5+ KB


### 4. Number of Rooms

### 5. Living Area

### 6. Kitchen

### 7. Furnished

### 8. Fireplace

### 9. Terrace

### 10. Terrace Area

### 11. Garden

### 12. Garden Area

### 13. Surface of the Land

### 14. Number of Facades

### 15. Swimming Pool

### 16. Building State